#### Here we will try to extract textual information from the Amazon Textract data

In [1]:
import pandas as pd
import json

Normalizing the json data 

In [2]:
json_filePath = 'Esempio ordini\\CAMBIO CASSONE SPAZIO SOC COOP PISTOIA 19.01.2022\\analyzeDocResponse.json'
with open(json_filePath) as json_file:
    data = json.load(json_file)

In [3]:
data.keys()

dict_keys(['Bucket', 'UploadedFileName', 'DocumentMetadata', 'JobStatus', 'Blocks', 'AnalyzeDocumentModelVersion'])

In [4]:
str(data.values())[:350]

"dict_values(['textract-console-us-east-1-86fe6673-b3e0-47d8-b007-e5ce570ce0f6', '8f5221b5_6c4d_4212_9a9a_6ecc919ef96b_cambio_cassone_spazio_soc_coop_pistoia_19.01.2022.pdf', {'Pages': 2}, 'SUCCEEDED', [{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 1, 'Height': 1, 'Left': 0, 'Top': 0}, 'Polygon': [{'X': 1.73299076420408e-16, 'Y': 0}, {"

we observe that we are getting key-words like - 
'Pages':
    --2
'BlockType':
    --'PAGE' 
'Geometry': 
    --'BoundingBox': {'Width': 1, 'Height': 1, 'Left': 0, 'Top': 0} 
    --'Polygon': [{'X': 1.73299076420408e-16, 'Y': 0}, {'X': 1, 'Y': 8.654170694555159e-17}, {'X': 1, 'Y': 1}, {'X': 0, 'Y': 1}]

#### Textract Response Parser

In [5]:
!pip install amazon-textract-response-parser

In [10]:
# data holds the Textract JSON dict
from trp.trp2 import TDocument, TDocumentSchema
t_doc = TDocumentSchema().dump(data)

### Pipelining now

In [11]:
# the sample code below makes use of the amazon-textract-caller
!pip install amazon-textract-caller

In [19]:
t_doc.keys()

dict_keys([])

In [13]:
from trp.t_pipeline import order_blocks_by_geo
import trp
import json

# the ordered_doc has elements ordered by y-coordinate (top to bottom of page)
ordered_doc = order_blocks_by_geo(t_doc)
# send to trp for further processing logic
trp_doc = trp.Document(TDocumentSchema().dump(ordered_doc))

AttributeError: 'dict' object has no attribute 'pages'

Deserialize Textract JSON

In [ ]:
from trp import Document
doc = Document(response)

# Iterate over elements in the document
for page in doc.pages:
    # Print lines and words
    for line in page.lines:
        print("Line: {}--{}".format(line.text, line.confidence))
        for word in line.words:
            print("Word: {}--{}".format(word.text, word.confidence))

    # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}-{}".format(r, c, cell.text, cell.confidence))

    # Print fields
    for field in page.form.fields:
        print("Field: Key: {}, Value: {}".format(field.key.text, field.value.text))

    # Get field by key
    key = "Phone Number:"
    field = page.form.getFieldByKey(key)
    if(field):
        print("Field: Key: {}, Value: {}".format(field.key, field.value))

    # Search fields by key
    key = "address"
    fields = page.form.searchFieldsByKey(key)
    for field in fields:
        print("Field: Key: {}, Value: {}".format(field.key, field.value))

### Exploring the keyValues file

In [4]:
dfKeyValues = pd.read_csv('Esempio ordini\CAMBIO CASSONE SPAZIO SOC COOP PISTOIA 19.01.2022\keyValues.csv')
print(dfKeyValues.shape)
dfKeyValues.head(5)

(36, 2)


,key,value
0,'PRODUTTORE,'SPAZIO SOC COOP
1,'CLIENTE,'SPAZIO SOC COOP
2,'Unit. Loc.,"'Via S. Agostino, 2 51100 PISTOIA (PT)"
3,'Operatore,'Francesca Buffolino - Mob. 327 6613420
4,'Referente,'MICHELE


Exploring the queryAnswer file

In [6]:
dfQueryAnswers = pd.read_csv('Esempio ordini\CAMBIO CASSONE SPAZIO SOC COOP PISTOIA 19.01.2022\queryAnswers.csv')
print(dfQueryAnswers.shape)
dfQueryAnswers.head()

(0, 4)


,Query,Answer,Alias,Page


Exploring the rawText.txt files

In [7]:
rawText_path = 'Esempio ordini\\CAMBIO CASSONE SPAZIO SOC COOP PISTOIA 19.01.2022\\rawText.txt'
with open(rawText_path) as f:
    lines= f.readlines()
print(lines)

['\n', 'UROCORPORATION\n', 'CONSULENZAE GESTIONE RIFIUTI\n', 'PER AUTISTA\n', 'PER UFFICIO\n', 'PRODUTTORE\n', 'SPAZIO SOC COOP\n', 'CLIENTE\n', 'SPAZIO SOC COOP\n', 'Unit. Loc.\n', 'Via S. Agostino, 2 51100 PISTOIA (PT)\n', 'Operatore\n', 'Francesca Buffolino - Mob. 327 6613420\n', 'Referente\n', 'MICHELE\n', 'CF Produttore\n', '06721950480\n', 'Telefono\n', '3395833947\n', 'Email Cliente\n', 'magazzinologistical@gmail.com\n', 'Data Richiesta\n', '19/01/2022\n', 'Orario richiesta\n', '10:00\n', 'Email Produttore\n', 'magazzinologistical@gmail.com\n', 'ORARI\n', 'ZTL\n', 'Mattina\n', 'Dalle\n', '09:00\n', 'Alle\n', '13:00\n', 'PER LA LOGISTICA:\n', 'Pomeriggio\n', 'Dalle\n', '14:00\n', 'Alle\n', '17:00\n', 'SCHEDA FATTA DA MATILDE.\n', 'CHIAMARE PER COMUNICARE GIORNO E ORARIO.\n', 'VEICOLO RICHIESTO\n', 'PREFERIBILMENTE IL POMERIGGIO\n', "CHIAMARE MICHELE AL 339/5833947 UN'ORA PRIMA DI\n", 'ARRIVARE.\n', 'CAMBIO CASSONE 30MC COPERTO.\n', 'Tipologia rifiuto\n', 'CER\n', 'DESCRIZIONE\n',

Exploring Tables

In [8]:
table_file_path = 'Esempio ordini\\CAMBIO CASSONE SPAZIO SOC COOP PISTOIA 19.01.2022\\table-'
dfTable = []
for i in range(1,7):
    dfTable.append(pd.read_csv(table_file_path+str(i)+'.csv'))
for i in dfTable:
    print('='*90)
    print(i.shape, i.columns, end = '\n')
    print(i.head())


(6, 3) Index([''', ''PER UFFICIO ', 'Unnamed: 2'], dtype='object')
                    '                             'PER UFFICIO   Unnamed: 2
0           'CLIENTE                          'SPAZIO SOC COOP          NaN
1         'Operatore   'Francesca Buffolino - Mob. 327 6613420          NaN
2     'CF Produttore                              '06721950480          NaN
3     'Email Cliente            'magazzinologistical@gmail.com          NaN
4  'Email Produttore            'magazzinologistical@gmail.com          NaN
(5, 5) Index([''', ''PER ', ''AUTISTA ', ''.1', 'Unnamed: 4'], dtype='object')
                  '               'PER            'AUTISTA       '.1  \
0      'PRODUTTORE    'SPAZIO SOC COOP                    '        '   
1      'Unit. Loc.   'Via S. Agostino,    '2 51100 PISTOIA    '(PT)    
2       'Referente            'MICHELE                    '        '   
3        'Telefono         '3395833947                    '        '   
4  'Data Richiesta         '19/01/2022

In [21]:
r, c =0,0
for i in range(len(dfTable)):
    r_,c_ = dfTable[i].shape
    r +=r_
    c += c_
    print(dfTable[i].shape)
print(r, c)

(6, 3)
(5, 5)
(2, 5)
(2, 5)
(2, 5)
(8, 4)
25 27


Figuring out the information from rest of the tables. 

In [30]:
net_keys = list(dfKeyValues.key)

### Perfroming NER on textual data

In [82]:
# rawText_path
with open(rawText_path, 'r') as file:
    lines = file.read().replace('\n', ' ')
with open(rawText_path, 'r') as file:
    lines_ = file.read().replace('\n', '###')

print(lines[:100],'\n', lines_[:100])

 UROCORPORATION CONSULENZAE GESTIONE RIFIUTI PER AUTISTA PER UFFICIO PRODUTTORE SPAZIO SOC COOP CLIE 
 ###UROCORPORATION###CONSULENZAE GESTIONE RIFIUTI###PER AUTISTA###PER UFFICIO###PRODUTTORE###SPAZIO S


In [83]:
lines_list = lines_.split('###')
lines_list[:10]

['',
 'UROCORPORATION',
 'CONSULENZAE GESTIONE RIFIUTI',
 'PER AUTISTA',
 'PER UFFICIO',
 'PRODUTTORE',
 'SPAZIO SOC COOP',
 'CLIENTE',
 'SPAZIO SOC COOP',
 'Unit. Loc.']

In [39]:
!pip install spacy
import spacy 
spacy.prefer_gpu()

False

In [42]:
!python -m spacy download it_core_news_sm


✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [46]:
!pip install prettytable
from prettytable import PrettyTable

### POS Tagging

In [48]:
t = PrettyTable(['token.text', 'token.lemma_', 'token.pos_', 'token.tag_', 'token.dep_',
            'token.shape_', 'token.is_alpha', 'token.is_stop'])

In [84]:
nlp_it = spacy.load("it_core_news_sm")
doc = nlp_it(lines)
for token in doc:
    t.add_row([token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop])
print(t)


+-------------------------------+-------------------------------+------------+------------+------------+------------------+----------------+---------------+
|           token.text          |          token.lemma_         | token.pos_ | token.tag_ | token.dep_ |   token.shape_   | token.is_alpha | token.is_stop |
+-------------------------------+-------------------------------+------------+------------+------------+------------------+----------------+---------------+
|                               |                               |   SPACE    |    _SP     |    dep     |                  |     False      |     False     |
|         UROCORPORATION        |         UROCORPORATION        |   PROPN    |     SP     |    ROOT    |       XXXX       |      True      |     False     |
|          CONSULENZAE          |          CONSULENZAE          |   PROPN    |     SP     | flat:name  |       XXXX       |      True      |     False     |
|            GESTIONE           |            GESTIONE     

C:\Users\sande\AppData\Local\Temp/ipykernel_10460/463249936.py:1: DeprecationWarning: [W107] The property `Doc.is_sentenced` is deprecated. Use `Doc.has_annotation("SENT_START")` instead.
  doc.is_sentenced


True

### NER tagging

In [85]:
ner_t = PrettyTable(['ent.text','ent.label_'])
for ent in doc.ents:
    ner_t.add_row([ent.text, ent.label_])
print(ner_t)

+------------------------------------------------------------------------------+------------+
|                                   ent.text                                   | ent.label_ |
+------------------------------------------------------------------------------+------------+
|                                UROCORPORATION                                |    MISC    |
|                               GESTIONE RIFIUTI                               |    MISC    |
|                                   AUTISTA                                    |    MISC    |
|                              UFFICIO PRODUTTORE                              |    ORG     |
|                               SPAZIO SOC COOP                                |    MISC    |
|                             SPAZIO SOC COOP Unit                             |    MISC    |
|                                 S. Agostino                                  |    PER     |
|                                   PISTOIA                 

In [87]:
from spacy import displacy
displacy.serve(doc, style = "ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [88]:
### NER on the values obtained from the key values pair csv file

In [105]:
str_keyValue = str(dfKeyValues.value)
str_keyValue= str_keyValue.replace(' ','')
str_keyValue

"0'SPAZIOSOCCOOP\n1'SPAZIOSOCCOOP\n2'ViaS.Agostino,251100PISTOIA(PT)\n3'FrancescaBuffolino-Mob.3276613420\n4'MICHELE\n5'06721950480\n6'3395833947\n7'\n8'magazzinologistical@gmail.com\n9'19/01/2022\n10'10:00\n11'\n12'magazzinologistical@gmail.com\n13'\n14'NOT_SELECTED\n15'\n16'09:00\n17'13:00\n18'\n19'SCHEDAFATTADAMATILDE.CHIAMAREPERCOMUNIC...\n20'\n21'\n22'14:00\n23'17:00\n24'\n25'\n26'\n27'NOT_SELECTED\n28'NOT_SELECTED\n29'\n30'Solidononpulverulento\n31'NOT_SELECTED\n32'\n33'R13\n34'CASSONE30MC\n35'EUROCORPORATION\nName:value,dtype:object"

In [119]:
import re
re_removeNext_charDigit = '\\n[0-9]*|\''
str_keyValue_ = re.sub(re_removeNext_charDigit, ' ', str_keyValue)
str_keyValue_

'0 SPAZIOSOCCOOP  SPAZIOSOCCOOP  ViaS.Agostino,251100PISTOIA(PT)  FrancescaBuffolino-Mob.3276613420  MICHELE  06721950480  3395833947    magazzinologistical@gmail.com  19/01/2022  10:00    magazzinologistical@gmail.com    NOT_SELECTED    09:00  13:00    SCHEDAFATTADAMATILDE.CHIAMAREPERCOMUNIC...      14:00  17:00        NOT_SELECTED  NOT_SELECTED    Solidononpulverulento  NOT_SELECTED    R13  CASSONE30MC  EUROCORPORATION Name:value,dtype:object'

In [120]:
ner_t = PrettyTable(['ent.text','ent.label_'])
doc = nlp_it(str_keyValue_)
for ent in doc.ents:
    ner_t.add_row([ent.text, ent.label_])
print(ner_t)

+-----------------------------------+------------+
|              ent.text             | ent.label_ |
+-----------------------------------+------------+
|         0 SPAZIOSOCCOOP           |    LOC     |
|           SPAZIOSOCCOOP           |    LOC     |
| FrancescaBuffolino-Mob.3276613420 |    PER     |
|              MICHELE              |    ORG     |
|       Solidononpulverulento       |    LOC     |
|            CASSONE30MC            |    LOC     |
|               dtype:              |    MISC    |
+-----------------------------------+------------+


In [122]:
displacy.serve(doc, style = "ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


# Trying hugging face for the images

In [9]:
#downloads transformers with tensorflow
!pip install git+https://github.com/huggingface/transformers


  Cloning https://github.com/huggingface/transformers to c:\users\sande\appdata\local\temp\pip-req-build-s2ck7yqc
  Resolved https://github.com/huggingface/transformers to commit d6b8e9cec7301ba02f642588a6f12e78ec3b9798
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for transformers: filename=transformers-4.20.0.dev0-py3-none-any.whl size=4226968 sha256=7fab70527eddee3be2544a23d278f922ea56f2691d835015f0f8380268dc1118
  Stored in directory: C:\Users\sande\AppData\Local\Temp\pip-ephem-wheel-cache-k240ho4z\wheels\14\a0\7b\8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.19.2
    Uninstalling tran

  Running command git clone -q https://github.com/huggingface/transformers 'C:\Users\sande\AppData\Local\Temp\pip-req-build-s2ck7yqc'


In [10]:
# checking whether the transformer works fine/not
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "c:\ProgramData\Anaconda3\lib\site-packages\transformers\__init__.py", line 30, in <module>
    from . import dependency_versions_check
  File "c:\ProgramData\Anaconda3\lib\site-packages\transformers\dependency_versions_check.py", line 17, in <module>
    from .utils.versions import require_version, require_version_core
  File "c:\ProgramData\Anaconda3\lib\site-packages\transformers\utils\__init__.py", line 46, in <module>
    from .hub import (
  File "c:\ProgramData\Anaconda3\lib\site-packages\transformers\utils\hub.py", line 39, in <module>
    from huggingface_hub import HfFolder, Repository, create_repo, list_repo_files, whoami
  File "c:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\__init__.py", line 70, in <module>
    from .keras_mixin import (
  File "c:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\keras_mixin.py", line 27, in <module>
    import tensorflow as tf
  File "c:\P

In [2]:
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

In [4]:
from PIL import Image

image = Image.open(
    "converted_pdf2img\\00.jpg"
)

width, height = image.size
print(image.size)

(1190, 1684)


In [5]:
from transformers import LayoutLMv2FeatureExtractor, LayoutLMv2TokenizerFast, LayoutLMv2Processor

feature_extractor = LayoutLMv2FeatureExtractor()  # apply_ocr is set to True by default
tokenizer = LayoutLMv2TokenizerFast.from_pretrained("microsoft/layoutlmv2-base-uncased")
processor = LayoutLMv2Processor(feature_extractor, tokenizer)

ModuleNotFoundError: No module named 'transformers'

In [1]:
!pip install -U label-studio

  Using cached label_studio-1.4.1.post1-py3-none-any.whl (42.8 MB)
  Using cached django_user_agents-0.4.0-py3-none-any.whl (8.6 kB)
  Using cached drf_yasg-1.20.0-py2.py3-none-any.whl (1.6 MB)
  Using cached rq-1.10.0-py2.py3-none-any.whl (68 kB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
  Using cached Django-3.1.14-py3-none-any.whl (7.8 MB)
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)
  Using cached sentry_sdk-1.5.12-py2.py3-none-any.whl (145 kB)
  Using cached django_filter-2.4.0-py3-none-any.whl (73 kB)
  Using cached xmljson-0.2.0-py2.py3-none-any.whl (14 kB)
  Using cached django_rest_swagger-2.2.0-py2.py3-none-any.whl (495 kB)
  Using cached django_rq-2.4.1-py2.py3-none-any.whl (45 kB)
  Using cached grpc_google_iam_v1-0.12.3-py3-none-any.whl
  Using cached google_cloud_logging-2.7.1-py2.py3-none-any.whl (173 kB)
  Using cached django_annoying-0.10.6-py2.py3-none-any.whl (9.5 kB)
  Using cached drf_generators-0.3.0-py2.py3-none-any.whl
  Usin

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pytz-2021.3-py3.10.egg-info\\dependency_links.txt'
Consider using the `--user` option or check the permissions.



  Using cached redis-4.3.3-py3-none-any.whl (244 kB)
  Using cached psycopg2_binary-2.9.1-cp39-cp39-win_amd64.whl (1.2 MB)
  Using cached boxing-0.1.4-py3-none-any.whl (3.9 kB)
  Using cached google_api_core-1.31.5-py2.py3-none-any.whl (93 kB)
  Using cached attr-0.3.1-py3-none-any.whl
  Using cached google_cloud_appengine_logging-1.1.0-py2.py3-none-any.whl (14 kB)
  Using cached googleapis_common_protos-1.52.0-py2.py3-none-any.whl (100 kB)
  Using cached drf_flex_fields-0.9.5-py3-none-any.whl
  Using cached boto3-1.16.63-py2.py3-none-any.whl (130 kB)
  Using cached label_studio_converter-0.0.39-py3-none-any.whl (26 kB)
  Using cached label_studio_tools-0.0.0.dev11-py3-none-any.whl (9.8 kB)
  Using cached google_resumable_media-0.5.1-py2.py3-none-any.whl (38 kB)
  Using cached google_cloud_storage-1.29.0-py2.py3-none-any.whl (85 kB)
  Using cached django_extensions-3.1.0-py3-none-any.whl (222 kB)
  Using cached django_ranged_fileresponse-0.1.2-py3-none-any.whl
  Using cached django_de

In [3]:
from studio-label-ocr.train import ocr_tasks.json

SyntaxError: invalid syntax (Temp/ipykernel_6940/1538263509.py, line 1)

In [8]:
#installing tensorflow
!nvidia-smi

Fri Jun 10 14:06:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.76       Driver Version: 496.76       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   69C    P8     8W /  N/A |    602MiB /  6144MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: c:\ProgramData\Anaconda3

  added / updated specs:
    - cudatoolkit=11.2
    - cudnn=8.1.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.2.2         |      h933977f_10       879.9 MB  conda-forge
    cudnn-8.1.0.77             |       h3e0f4f4_0       610.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        1.46 GB

The following NEW packages will be INSTALLED:

  cudatoolkit        conda-forge/win-64::cudatoolkit-11.2.2-h933977f_10
  cudnn              conda-forge/win-64::cudnn-8.1.0.77-h3e0f4f4_0




cudatoolkit-11.2.2   | 879.9 MB  |            |   0% 
cudatoolkit-11.2.2   | 879.9 MB  |            |



==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 4.13.0

Please update conda by running

    $ conda update -n base conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: c:\ProgramData\Anaconda3




In [12]:
!pip install --upgrade pip

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pip\\py.typed'



  Using cached pip-22.1.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:


Consider using the `--user` option or check the permissions.



In [18]:
# !pip install tensorflow
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

ImportError: Could not find the DLL(s) 'msvcp140_1.dll'. TensorFlow requires that these DLLs be installed in a directory that is named in your %PATH% environment variable. You may install these DLLs by downloading "Microsoft C++ Redistributable for Visual Studio 2015, 2017 and 2019" for your platform from this URL: https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads

In [3]:
torch.cuda.is_available()

False

In [4]:
!pip install tensorflow
import tensorflow as tf
tf.config.list_physical_devices('GPU')

ImportError: Could not find the DLL(s) 'msvcp140_1.dll'. TensorFlow requires that these DLLs be installed in a directory that is named in your %PATH% environment variable. You may install these DLLs by downloading "Microsoft C++ Redistributable for Visual Studio 2015, 2017 and 2019" for your platform from this URL: https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads

In [6]:
tf.version.VERSION

NameError: name 'tf' is not defined

In [1]:
!label-studio

^C


=> Database and media directory: C:\Users\sande\AppData\Local\label-studio\label-studio
=> Static URL is set to: /static/
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /pypi/label-studio/json HTTP/1.1" 200 46133
[2022-06-07 14:23:19,705] [ldclient.util::all_flags_state::367] [WARNING] all_flags_state() called, but client is in offline mode. Returning empty state
[07/Jun/2022 14:23:19] "GET /projects HTTP/1.1" 200 4084
[07/Jun/2022 14:23:19] "GET /static/css/uikit.e49a7a43adbd.css HTTP/1.1" 200 3892
[07/Jun/2022 14:23:19] "GET /static/css/main.de66392810e2.css HTTP/1.1" 200 15353
[07/Jun/2022 14:23:20] "GET /static/fonts/roboto/roboto.css HTTP/1.1" 200 3431
[07/Jun/2022 14:23:20] "GET /react-app/main.css?v=a26ba3 HTTP/1.1" 200 68374
[07/Jun/2022 14:23:20] "GET /static/js/helpers.9a01b074ff58.js HTTP/1.1" 200 13336
[07/Jun/2022 14:23:20] "GET /static/js/jquery.min.220afd743d9e.js HTTP/1.1" 200 88145
[07/Jun/2022 14:23:20] "GET /react-app/index.js?v=a26ba3 HTTP

Current platform is win32, apply sqlite fix
Can't load sqlite3.dll from current directory
Performing system checks...

System check identified no issues (0 silenced).
June 07, 2022 - 14:23:19
Django version 3.1.14, using settings 'label_studio.core.settings.label_studio'
Starting development server at http://0.0.0.0:8080/
Quit the server with CTRL-BREAK.


[07/Jun/2022 16:06:17] "GET /api/projects/10/summary HTTP/1.1" 200 162
[07/Jun/2022 16:06:17] "GET /api/projects/10 HTTP/1.1" 200 2568
[07/Jun/2022 16:06:17] "POST /api/projects/10/sample-task HTTP/1.1" 200 79
[07/Jun/2022 16:06:17] "GET /api/projects/10/summary HTTP/1.1" 200 162
[07/Jun/2022 16:06:17] "GET /api/projects/10 HTTP/1.1" 200 2568
[07/Jun/2022 16:06:18] "GET /api/projects/10 HTTP/1.1" 200 2568
[07/Jun/2022 16:06:18] "GET /api/templates HTTP/1.1" 200 33563
[07/Jun/2022 16:06:18] "GET /api/templates HTTP/1.1" 200 33563
[07/Jun/2022 16:06:19] "GET /api/templates HTTP/1.1" 200 33563
[07/Jun/2022 16:06:22] "GET /api/projects/10 HTTP/1.1" 200 2568
[07/Jun/2022 16:06:24] "GET /api/projects/10/model-versions HTTP/1.1" 200 16
[07/Jun/2022 16:06:24] "GET /api/projects/10 HTTP/1.1" 200 2568
[07/Jun/2022 16:06:24] "GET /api/ml?project=10 HTTP/1.1" 200 2
[07/Jun/2022 16:06:24] "GET /api/projects/10/model-versions HTTP/1.1" 200 16
[07/Jun/2022 16:06:24] "GET /api/ml?project=10 HTTP/1.1" 

In [7]:
def create_image_url(filepath):
    """
    Label Studio requires image URLs, so this defines the mapping from filesystem to URLs
    if you use ./serve_local_files.sh <my-images-dir>, the image URLs are localhost:8081/filename.png
    Otherwise you can build links like /data/upload/filename.png to refer to the files
    """
    filename = os.path.basename(filepath)
    # return f'http://localhost:8081/{filename}'
    return filename

create_image_url('Esempio ordini\\converted_pdf2img\\00.jpg')

'00.jpg'

In [45]:
from studio-label-ocr.train import tesseractocr.py

SyntaxError: invalid syntax (Temp/ipykernel_14360/1858271853.py, line 1)

In [28]:
from pathlib import Path
import glob

tasks = []
# collect the receipt images from the image directory
for f in Path('studio-label-ocr\\train\\').glob('*.jpg'): #Path('studio-label-ocr//train//')
    print(f)
    with Image.open(f.absolute()) as image:
        tesseract_output = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
        task = convert_to_ls(image, tesseract_output, per_level='block_num')
        tasks.append(task)
print(tasks)

studio-label-ocr\train\00.jpg


NameError: name 'Image' is not defined